In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

In [2]:
from liana.method.sp._Misty import MistyData
from liana.method.sp._misty_constructs import genericMistyData

In [3]:
adata = sc.read_h5ad('liana/tests/data/synthetic.h5ad')

In [4]:
misty = genericMistyData(intra=adata)

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [5]:
misty

MuData object with n_obs × n_vars = 4205 × 33
  obs:	'cell_type'
  3 modalities
    intra:	4205 x 11
      obs:	'cell_type'
      obsm:	'spatial'
    juxta:	4205 x 11
      obsm:	'spatial'
      obsp:	'spatial_connectivities'
    para:	4205 x 11
      obsm:	'spatial'
      obsp:	'spatial_connectivities'

In [6]:
misty(n_estimators=20)

In [7]:
misty.uns['target_metrics']

,target,intra_group,env_group,intra.R2,multi.R2,gain.R2,intra,juxta,para
0,ECM,None,None,0.915215,0.930767,0.015552,0.814531,0.172121,0.013348
1,ligA,None,None,0.988723,0.988971,0.000247,0.967293,0.028793,0.003914
2,ligB,None,None,0.930962,0.943334,0.012372,0.812734,0.158387,0.028879
3,ligC,None,None,0.772749,0.852782,0.080033,0.620814,0.322833,0.056353
4,ligD,None,None,0.975034,0.975947,0.000913,0.942298,0.050840,0.006862
5,protE,None,None,0.996774,0.996772,-0.000002,1.000000,0.000000,0.000000
6,protF,None,None,0.998864,0.998865,0.000002,0.995857,0.000863,0.003280
7,prodA,None,None,0.991412,0.991424,0.000012,0.998923,0.000000,0.001077
8,prodB,None,None,0.980528,0.980439,-0.000089,1.000000,0.000000,0.000000
9,prodC,None,None,0.954489,0.954813,0.000324,1.000000,0.000000,0.000000


In [8]:
misty.uns['importances']

,target,predictor,intra_group,env_group,view,value
0,ECM,ligA,None,None,intra,0.006981
1,ECM,ligB,None,None,intra,0.022493
2,ECM,ligC,None,None,intra,0.068925
3,ECM,ligD,None,None,intra,0.010800
4,ECM,protE,None,None,intra,0.359283
...,...,...,...,...,...,...
325,prodD,protE,None,None,para,0.103775
326,prodD,protF,None,None,para,0.139170
327,prodD,prodA,None,None,para,0.083315
328,prodD,prodB,None,None,para,0.108542


lrMistyData

In [9]:
from liana.testing._sample_anndata import generate_toy_spatial

In [10]:
from liana.resource import select_resource
from liana.method._pipe_utils import prep_check_adata, assert_covered
from liana.method._pipe_utils._pre import _get_props

from liana.method.sp._SpatialMethod import _SpatialMeta
from liana.method.sp._spatial_pipe import _get_ordered_matrix, _rename_means, \
    _run_scores_pipeline, _connectivity_to_weight, _handle_connectivity, _categorize
from liana.method.sp._bivariate_funs import _handle_functions

In [11]:
from liana.method.sp._lr_basis import _add_complexes_to_var

In [12]:
from liana.method.sp._misty_constructs import _make_view

In [13]:
from liana.method.sp._spatial_pipe import spatial_neighbors

In [14]:
adata = generate_toy_spatial()

In [15]:
use_raw = False
layer = None
verbose = True
resource = None
resource_name = 'consensus'
nz_threshold = 0.1

In [16]:
spatial_key = 'spatial'
bandwidth = 100
kernel = 'misty_rbf'
set_diag = True
cutoff = 0.1
zoi = 0

In [17]:
if resource is None:
    resource = select_resource(resource_name.lower())

In [18]:
views = dict()

In [19]:
adata

AnnData object with n_obs × n_vars = 700 × 765
    obs: 'bulk_labels', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'louvain'
    var: 'n_counts', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'bulk_labels_colors', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'distances', 'connectivities', 'spatial_connectivities'

In [20]:
adata = prep_check_adata(adata=adata,
                         use_raw=use_raw,
                         layer=layer,
                         verbose=verbose,
                         groupby=None,
                         min_cells=None,
                         obsm = {spatial_key: adata.obsm[spatial_key]}
                         )

Using `.X`!
Converting mat to CSR format


In [21]:
adata = _add_complexes_to_var(adata,
                              np.union1d(resource['receptor'].astype(str),
                                         resource['ligand'].astype(str)
                                        )
                              )

In [22]:
adata

AnnData object with n_obs × n_vars = 700 × 769
    obs: 'bulk_labels', 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'louvain'
    obsm: 'spatial'
    obsp: 'distances', 'connectivities', 'spatial_connectivities'

In [23]:
# filter_resource
resource = resource[(np.isin(resource.ligand, adata.var_names)) &
                    (np.isin(resource.receptor, adata.var_names))]

In [24]:
intra = _make_view(adata=adata[:, resource['receptor'].unique()],
                   nz_threshold=nz_threshold, obs=adata.obs)

views['intra'] = _add_complexes_to_var(intra, np.unique(resource['receptor']))

In [25]:
connectivity = spatial_neighbors(adata=adata, spatial_key=spatial_key,
                                 bandwidth=bandwidth, kernel=kernel,
                                 set_diag=set_diag, cutoff=cutoff,
                                 zoi=zoi, inplace=False)

extra = _make_view(adata=adata[:,resource['ligand'].unique()], spatial_key=spatial_key,
                   nz_threshold=nz_threshold, connecitivity=connectivity, obs=pd.DataFrame(index=adata.obs.index))
views['extra'] = _add_complexes_to_var(extra, np.unique(resource['ligand']))

In [26]:
from mudata import MuData

In [34]:
lr_misty = MistyData(data=views, obs=intra.obs, spatial_key=spatial_key)

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [35]:
lr_misty()

ValueError: Length of values (15) does not match length of index (26)